In [ ]:
from __future__ import division
%matplotlib inline
import CameraNetwork
import CameraNetwork.global_settings as gs
import cPickle
import cv2
import datetime
import Image
import ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import tqdm
from tqdm import tqdm_notebook
import traceback
plt.rcParams['figure.figsize'] = (8.0, 8.0)

i:\amit\office backup\amit\studies\cameranetwork_git\CameraNetwork\transformation_matrices.py:1899: UserWarning: failed to import module _transformations
  warnings.warn("failed to import module %s" % name)


In [ ]:
c = CameraNetwork.CLIclient()
proxy_params = CameraNetwork.retrieve_proxy_parameters()
c.start(proxy_params)
time.sleep(3)
print c.client_instance.servers

In [ ]:
dfs = {}
tnb1 = tqdm_notebook(c.client_instance.servers, leave=False)
for server in tnb1:
    tnb1.set_description("Server {}".format(server))
    tnb1.refresh()
    days_list = c.days(server)[1][0]["days_list"]
    tmp_dfs = []
    tnb2 = tqdm_notebook(days_list, leave=False)
    for day in tnb2:
        try:
            tnb2.set_description(day)
            tnb2.refresh()
            tmp_dfs.append(c.query(server_id=server, query_day=datetime.datetime.strptime(day, "%Y_%m_%d").date())[0])
        except:
            tqdm.tqdm.write("Failed loading day {} of server {}".format(day, server))
    dfs[server] = pd.concat(tmp_dfs, axis=0)

In [ ]:
dfs.keys()

In [ ]:
with open("thumbnails.pkl", "wb") as f:
    cPickle.dump(dfs, f)